In [17]:
##17 hashes are duplicated i.e, have duplicate URLs and have been downled as a same report with same TTPs as two different hashes -> 120 - 17 is the true ground truth
# Function to find unique hashes with URLs and track removed hashes
import json
import os

In [18]:
# Function to count the number of URLs in the file
def count_urls(file_path):
    url_count = 0
    
    # Open and read the file
    with open(file_path, 'r') as file:
        for line in file:
            # Count non-empty lines (assuming each line contains one URL)
            if line.strip():  # Strip any whitespace and check if the line is not empty
                url_count += 1
                
    return url_count

file_path = r"C:\Users\Aakanksha Saha\Documents\CTI_downloads\downloads\20240930_urls.txt"

# Call the function and print the result
print(f'Total number of URLs: {count_urls(file_path)}')

Total number of URLs: 1372


In [44]:
import os

# Function to list and count files in a folder, excluding certain file types
def list_downloaded_files(folder_path):
    downloaded_files = []

    # Walk through the directory structure
    for root, dirs, files in os.walk(folder_path):
        for file_name in files:
            # Exclude files ending with '.download.iocs'
            if not file_name.endswith('.download.iocs'):
                downloaded_files.append(file_name)

    # Return the list of files and the total count
    return downloaded_files, len(downloaded_files)

# Specify the path to the folder containing downloaded files
#folder_path_downloads = r"C:\Users\Aakanksha Saha\Documents\CTI_downloads\downloads\20241008_downloads"
folder_path_downloads = 'C:/Users/Aakanksha Saha/Documents/CTI_downloads/malpedia_20220718/malpedia_20220718/documents'

# Call the function and get the list of files and their count
downloaded_files, total_count = list_downloaded_files(folder_path_downloads)

# Print the list of files and the total count
print("List of downloaded files (SHA256 hash names):")
#for file_name in downloaded_files:
#    print(file_name)

print(f"\nTotal number of downloaded files: {total_count}")


List of downloaded files (SHA256 hash names):

Total number of downloaded files: 11339


In [45]:
import json

# Function to identify successful downloads, filtering by download size
def identify_successful_downloads(file_path):
    successful_downloads = []
    
    # Open and read the file line by line
    with open(file_path, 'r') as file:
        for line in file:
            try:
                # Parse each line as a JSON object
                download_info = json.loads(line)
                
                # Check if the download status is 200 (successful) and size is 20KB or more
                if download_info.get("download_status") == 200:
                #and download_info.get("download_size", 0) >= 20000:
                    successful_downloads.append({
                        "url": download_info.get("url"),
                        "download_sha256": download_info.get("download_sha256"),
                        "download_status": download_info.get("download_status"),
                        "download_size": download_info.get("download_size"),
                        "download_ts": download_info.get("download_ts")
                    })
            except json.JSONDecodeError:
                # Handle case where the line is not a valid JSON (optional)
                print(f"Error decoding JSON for line: {line.strip()}")
    
    return successful_downloads

In [46]:
# Call the function and get the successful downloads
successful_downloads = identify_successful_downloads(file_path_for_jsonl)

# Print the total count of successful downloads with size >= 20KB
print(f"\nTotal number of successful downloads (size >= 20KB): {len(successful_downloads)}")


Total number of successful downloads (size >= 20KB): 11385


In [47]:
##Extracting report hashes from the filenames
# Function to recursively extract the hash part of filenames ending with '.download.iocs'
def extract_hashes_from_filenames(folder_path_of_iocs):
    file_hash_list = []

    # Walk through the directory structure recursively
    for root, dirs, files in os.walk(folder_path_of_iocs):
        for file_name in files:
            # Check if the filename ends with '.download.iocs'
            if file_name.endswith('.download.iocs'):
                # Extract the hash part (the filename before the extension)
                hash_part = file_name.split('.download.iocs')[0]
                file_hash_list.append(hash_part)

    return file_hash_list

In [48]:
folder_path_of_iocs = 'C:/Users/Aakanksha Saha/Documents/CTI_downloads/malpedia_20220718/malpedia_20220718/iocs'
#folder_path_of_iocs = 'C:/Users/Aakanksha Saha/Documents/CTI_downloads/downloads/20241008_downloads/iocs2'
# Extract the hashes from filenames and store them in a list
file_hash_list = extract_hashes_from_filenames(folder_path_of_iocs)

# Print the total number of hashes
print(f"\nTotal file hashes extracted: {len(file_hash_list)}")


Total file hashes extracted: 1051


In [49]:
#file_path_for_jsonl = r"C:\Users\Aakanksha Saha\Documents\CTI_downloads\downloads\20241008_downloads.jsonl"
file_path_for_jsonl = r"C:\Users\Aakanksha Saha\Documents\CTI_downloads\malpedia_20220718\malpedia_20220718\malpedia-db_2022-07-18_downloader.jsonl"

In [50]:
def find_hashes_grouped_by_url(file_path_for_jsonl, file_hash_list):
    hash_url_dict = {}  # Dictionary to store URLs as keys, and hashes as values (list)
    removed_hashes = {}  # To store removed hashes and their duplicated URLs
    seen_urls = set()    # Set to track URLs and remove duplicates

    # Open and read the file line by line
    with open(file_path_for_jsonl, 'r') as file:
        for line in file:
            try:
                # Parse each line as a JSON object
                download_info = json.loads(line)

                # Get the hash and url from the JSON line
                sha256_hash = download_info.get("download_sha256")
                url = download_info.get("url")

                # Check if the download_sha256 is in the provided hash list
                if sha256_hash in file_hash_list:
                    # Only process hashes with unique URLs
                    if url not in seen_urls:
                        # If the URL is new, create an entry in the dictionary
                        hash_url_dict[url] = [sha256_hash]
                        seen_urls.add(url)  # Mark the URL as seen
                    else:
                        # If the URL is duplicated, add the hash to the URL's list
                        hash_url_dict[url].append(sha256_hash)

                        # Track the removed hash if it's a duplicate for that URL
                        removed_hashes[sha256_hash] = url
                        file_hash_list.remove(sha256_hash)

            except json.JSONDecodeError:
                # Handle case where the line is not a valid JSON
                print(f"Error decoding JSON for line: {line.strip()}")
    
    return hash_url_dict, removed_hashes


In [51]:
# Call the function and retrieve the hash-to-URL dictionary and removed hashes
hash_url_dict, removed_hashes = find_hashes_grouped_by_url(file_path_for_jsonl, file_hash_list)

# Print the resulting dictionary of hashes grouped by URL
#print("Hashes grouped by URL:")
#for url, hashes in hash_url_dict.items():
#    print(f"URL: {url}")
#    for sha256_hash in hashes:
#        print(f"    Hash: {sha256_hash}")

# Print the total number of unique hash groups (URLs)
print(f"\nTotal unique hash groups: {len(hash_url_dict)}")

# Print the removed hashes and their associated URLs
#print("\nRemoved hashes due to duplicate URLs:")
#for sha256_hash, url in removed_hashes.items():
#    print(f"Removed Hash: {sha256_hash}, URL: {url}")

# Print the total number of removed hashes
print(f"\nTotal removed hashes: {len(removed_hashes)}")


Total unique hash groups: 1055

Total removed hashes: 0
